In [21]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# 1. Read Original Data 

## 1.1 Load Original Dataset

Remember to change the path to read data

In [22]:
df_problem = pd.read_csv('./Data/round1-day1_problem_data.csv')
burrito_price = float(df_problem['burrito_price'][0])
ingredient_cost = float(df_problem['ingredient_cost'][0])
truck_cost = float(df_problem['truck_cost'][0])
print(f"  - The burritos cost ₲{ingredient_cost} to make and are sold for ₲{burrito_price}. Each truck costs ₲{truck_cost} to use per day.")

df_truck_node = pd.read_csv('./Data/round1-day1_truck_node_data.csv')
truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in df_truck_node.iterrows()}
truck_spots = truck_coordinates.keys()
print(f"  - There are {len(truck_spots)} available 'truck_spots' or places where a truck can be placed around Burritoville.")

df_demand = pd.read_csv('./Data/round1-day1_demand_node_data.csv')
buildings, building_names, building_coordinates, demand = gp.multidict({
        row['index']: [row['name'], (float(row['x']), float(row['y'])), float(row['demand'])] for ind,row in df_demand.iterrows()
    })
print(f"  - There are in {len(buildings)} buildings with hungry customers also known as demand nodes.")

df_truck_demand_pair = pd.read_csv('./Data/round1-day1_demand_truck_data.csv')
building_truck_spot_pairs, distance, scaled_demand = gp.multidict({
        (row['demand_node_index'], row['truck_node_index']): [float(row['distance']), float(row['scaled_demand'])] for ind,row in df_truck_demand_pair.iterrows() if float(row['scaled_demand'])>0# (building, truck_spot): distance, scaled_demand
    })
print(f"  - There are in {len(building_truck_spot_pairs)} pairs of trucks spots and buildings with hungry customers.")




  - The burritos cost ₲5.0 to make and are sold for ₲10.0. Each truck costs ₲250.0 to use per day.
  - There are 15 available 'truck_spots' or places where a truck can be placed around Burritoville.
  - There are in 15 buildings with hungry customers also known as demand nodes.
  - There are in 100 pairs of trucks spots and buildings with hungry customers.


In [23]:
# 创建 vehicle_type 的列表
vehicle_types = pd.DataFrame({'vehicle_type': [0, 1, 2]})

# 使用 merge 进行交叉连接（类似 SQL 的 CROSS JOIN）
df_truck_expanded = df_truck_node.merge(vehicle_types, how='cross')

# 重新安排列顺序
df_truck_expanded=df_truck_expanded[['index', 'vehicle_type', 'x', 'y']]

In [24]:
df_truck_expanded

,index,vehicle_type,x,y
0,truck1,0,112.755064,58.066778
1,truck1,1,112.755064,58.066778
2,truck1,2,112.755064,58.066778
3,truck6,0,56.222222,129.485810
4,truck6,1,56.222222,129.485810
5,truck6,2,56.222222,129.485810
6,truck7,0,113.065684,131.038397
7,truck7,1,113.065684,131.038397
8,truck7,2,113.065684,131.038397
9,truck8,0,164.628607,130.417362


## 1.2 Drawing the map

In [25]:
import plotly.graph_objects as go
from PIL import Image
import requests
from io import BytesIO #try 1

def show_map(buildings, building_names, building_coordinates, demand, truck_coordinates, placed_trucks = []):
    """displays the Burrito Optimization map with labels for open truck locations, buildings with demand, and placed trucks [optional].  This is intended to be used in the Gurobi Days Intro to Modeling course"""

    y_max = 550

    truck_spot_x = [value[0] for key, value in truck_coordinates.items()]
    truck_spot_y = [y_max - value[1] for key, value in truck_coordinates.items()]
    trucks = [key for key, value in truck_coordinates.items()]

    building_x = [value[0] for key, value in building_coordinates.items()]
    building_y = [y_max - value[1] for key, value in building_coordinates.items()]
    demand = [value for key, value in demand.items()]
    building_names = [value for key, value in building_names.items()]

    if placed_trucks:
        placed_truck_spot_x = [value[0] for key, value in truck_coordinates.items() if key in placed_trucks]
        placed_truck_spot_y = [y_max - value[1] for key, value in truck_coordinates.items() if key in placed_trucks]
        placed_trucks = [key for key, value in truck_coordinates.items() if key in placed_trucks]

    map = Image.open('map.png')
    fig = go.Figure()

    # Add trace for truck spots
    fig.add_trace(
        go.Scatter(x=truck_spot_x, y=truck_spot_y,
                   hovertemplate=trucks,
                   name="Open truck spots",
                   mode='markers',
                   marker_color='rgba(135, 206, 250, 0.0)',
                   marker_line_color='darkblue',
                   marker_line_width=2,
                   marker_size=10
                   )
    )

    # Add trace for placed trucks
    if placed_trucks:
        fig.add_trace(
            go.Scatter(x=placed_truck_spot_x, y=placed_truck_spot_y,
                       hovertemplate=placed_trucks,
                       name="Truck added to the map",
                       mode='markers',
                       marker_color='darkblue',
                       marker_line_color='darkblue',
                       marker_line_width=2,
                       marker_size=10
                       )
        )

    # Add trace for buildings
    fig.add_trace(
        go.Scatter(x=building_x, y=building_y,
                   hovertemplate=building_names,
                   name="Buildings with customer demand",
                   mode='markers',
                   marker_color='red',
                   marker_opacity=0.5,
                   marker_line_width=0,
                   marker_size=demand
                   )
    )

    # Add minimap image
    fig.add_layout_image(
            dict(
                source=map,
                xref="x",
                yref="y",
                x=0,
                y=550,
                sizex=500,
                sizey=550,
                sizing="stretch",
                opacity=0.9,
                layer="below")
    )

    # Set templates
    fig.update_layout(template="simple_white")
    fig.update_xaxes(range=[0, 500], visible=False)
    fig.update_yaxes(range=[0,550], visible=False,
                    scaleanchor = "x",scaleratio = 1)
    fig.update_layout(showlegend=True)

    fig.update_layout(
        title="Burrito Optimization Game Map",
    )

    fig.show()



In [26]:
#from show_map_local import show_map

show_map(buildings, building_names, building_coordinates, demand, truck_coordinates)

# 2. Data Generation

## 2.1 Generate Preference Matrix

We regenerate the scaled_demand data(the deal between customer and truck) by make a preference matrix with a exp funtion of distance.

In [27]:
import math
def generate_scaled_demand(df_scaled,df_demand_node):
    """
    df_scaled: the demand-truck pairs dataset
    df_demand_node: the demand node dataset
    """
    df_rescaled = df_scaled.copy()

    #del(dict)
    my_dict = dict(zip(df_demand_node['index'], df_demand_node['demand']))
    df_rescaled['demand'] = df_rescaled['demand_node_index'].map(my_dict)

    # Step1:Calculate the total distance of each demand node for all truck node
    distance_sums = df_rescaled.groupby('demand_node_index')['distance'].transform('sum')

    # Step2:Calculate the value of new column
    beta = 10
    df_rescaled['scaled_demand'] = (np.exp(-beta * (df_rescaled['distance'] / distance_sums))) * df_rescaled['demand']
    df_rescaled['scaled_demand'] = df_rescaled['scaled_demand'].astype(int)

    df_rescaled = df_rescaled.drop(columns=['demand'])

    return df_rescaled

In [28]:
df_scaled = generate_scaled_demand(df_truck_demand_pair,df_demand)
df_scaled

,demand_node_index,truck_node_index,distance,scaled_demand
0,demand2,truck1,231.108640,20
1,demand2,truck6,201.157571,21
2,demand2,truck7,161.831991,22
3,demand2,truck8,113.778542,24
4,demand2,truck17,289.327111,18
...,...,...,...,...
220,demand51,truck36,298.355857,26
221,demand51,truck37,379.099609,22
222,demand51,truck43,408.808936,21
223,demand51,truck49,482.584417,18


# 3. LP Model and Solution

## 3.1 Load the Data

In [29]:
def parameter_read(df_problem,df_truck_node,df_demand,df_scaled):
    burrito_price = float(df_problem['burrito_price'][0])
    ingredient_cost = float(df_problem['ingredient_cost'][0])

    truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in df_truck_node.iterrows()}
    truck_spots = truck_coordinates.keys()

    buildings, building_names, building_coordinates, demand = gp.multidict({
        row['index']: [row['name'], (float(row['x']), float(row['y'])), float(row['demand'])] for ind,row in df_demand.iterrows()
    })

    building_truck_spot_pairs, distance, scaled_demand = gp.multidict({
        (row['demand_node_index'], row['truck_node_index']): [float(row['distance']), float(row['scaled_demand'])] for ind,row in df_scaled.iterrows() if float(row['scaled_demand'])>0# (building, truck_spot): distance, scaled_demand
    })

    return burrito_price, ingredient_cost,truck_coordinates,truck_spots,buildings,building_names,building_coordinates,demand,scaled_demand

In [30]:
# Read data in files
burrito_price, ingredient_cost,truck_coordinates,truck_spots,buildings,building_names,building_coordinates,demand,scaled_demand = parameter_read(df_problem,df_truck_node,df_demand,df_truck_demand_pair)

## 3.2 Build Up the LP Model

In [31]:
# ================== 扩展卡车类型数据 ==================
# 定义卡车类型属性 (0: small, 1: medium, 2: large)
truck_types = {
    0: {'capacity': 100, 'fixed_cost': 200},
    1: {'capacity': 150, 'fixed_cost': 250}, 
    2: {'capacity': 200, 'fixed_cost': 300}
}

def extended_LP_solution(burrito_price, ingredient_cost, truck_types,
                       truck_coordinates, truck_spots, buildings,
                       building_names, building_coordinates, demand,
                       building_truck_spot_pairs, distance, scaled_demand,
                       work_name="Extended_Solution", max_distance=500,
                       write_lp_file=True, show_model=True, is_show_map=True):
    """支持多卡车类型和可视化控制的完整优化模型"""
    # --- 初始化模型 ---
    model = gp.Model("Extended_Truck_Deployment")
    model.Params.OutputFlag = int(show_model)
    
    # --- 决策变量 ---
    x = model.addVars(truck_types.keys(), truck_spots, vtype=GRB.BINARY, name="x")
    y = model.addVars(building_truck_spot_pairs, vtype=GRB.BINARY, name="y")
    
    # --- 目标函数 ---
    revenue = gp.quicksum(
        (burrito_price - ingredient_cost) * scaled_demand[i,j] * y[i,j]
        for i,j in building_truck_spot_pairs
    )
    cost = gp.quicksum(
        truck_types[k]['fixed_cost'] * x[k,j]
        for k in truck_types for j in truck_spots
    )
    model.setObjective(revenue - cost, GRB.MAXIMIZE)
   # --- 完全规范的约束条件写法 ---

    # --- 约束条件 ---
    # 1. 每个客户最多被一个位置服务
    model.addConstrs(
        (y.sum(i,'*') <= 1 for i in buildings),
        name="Single_Service"
    )
    
    # 2. 服务激活约束：只有部署了卡车的点才能服务客户
    model.addConstrs(
        (y[i,j] <= x.sum('*',j) for i,j in building_truck_spot_pairs),
        name="Service_Activation"
    )
    
    # 3. 距离约束：服务距离不超过max_distance
    model.addConstrs(
        (y[i,j] * distance[i,j] <= max_distance for i,j in building_truck_spot_pairs),
        name="Max_Distance"
    )
    
    # 4. 容量约束：每个位置的总需求不超过卡车容量
    model.addConstrs(
        (gp.quicksum(
            scaled_demand[i,j] * y[i,j] 
            for i in buildings if (i,j) in building_truck_spot_pairs
        ) <= gp.quicksum(
            truck_types[k]['capacity'] * x[k,j] 
            for k in truck_types
        ) for j in truck_spots),
        name="Capacity"
    )
    
    # 5. 每个位置最多部署一种卡车类型
    model.addConstrs(
        (x.sum('*',j) <= 1 for j in truck_spots),
        name="Unique_Truck_Type"
    )
    
    # --- 模型输出 ---
    if write_lp_file:
        model.write(f"{work_name}.lp")
    
    # --- 求解 ---
    model.optimize()
    
    # --- 结果展示 ---
    if show_model and model.status == GRB.OPTIMAL:
        print("\n=== 最优解 ===")
        print(f"总利润: ₲{model.objVal:.2f}")
        
        # 卡车部署详情
        deployed_trucks = [(k,j) for k,j in x.keys() if x[k,j].X > 0.5]
        for k,j in deployed_trucks:
            truck_type = ['Small', 'Medium', 'Large'][k]
            print(f" - 在位置 {j} 部署 {truck_type} 卡车 (容量: {truck_types[k]['capacity']}, 成本: ₲{truck_types[k]['fixed_cost']})")
        
        # 财务明细
        print("\n财务明细:")
        print(f"总收入: ₲{revenue.getValue():.2f}")
        print(f"总成本: ₲{cost.getValue():.2f}")
    
    # --- 可视化 ---
    if is_show_map and model.status == GRB.OPTIMAL:
        placed_trucks = [j for k,j in x.keys() if x[k,j].X > 0.5]  # 获取所有部署位置
        show_map(buildings, building_names, building_coordinates, demand, 
                truck_coordinates, placed_trucks=placed_trucks)
    
    return model.objVal


## 3.3 Running the Model

In [32]:
# Solve the original problem
work_name = "Part 1 Original Problem"
# ================== 调用示例 ==================
print("\n正在求解扩展模型...")
optimal_value = extended_LP_solution(
    burrito_price, ingredient_cost, truck_types,
    truck_coordinates, truck_spots, buildings,
    building_names, building_coordinates, demand,
    building_truck_spot_pairs, distance, scaled_demand,
    work_name="Extended_Solution",
    max_distance=500,
    write_lp_file=True,
    show_model=True,
    is_show_map=True
)


正在求解扩展模型...
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 245 rows, 145 columns and 790 nonzeros
Model fingerprint: 0x6a0fb15f
Variable types: 0 continuous, 145 integer (145 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 126 rows and 30 columns
Presolve time: 0.00s
Presolved: 119 rows, 115 columns, 360 nonzeros
Variable types: 0 continuous, 115 integer (115 binary)

Root relaxation: objective 7.528571e+02, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  

H    0     0                     620.0000000  752.85714  21.4%     -    0s
H    0     0                     655.0000000  752.85714  14.9%     -    0s
H    0     0                     700.0000000  752.85714  7.55%     -    0s
H    0     0                     715.0000000  752.85714  5.29%     -    0s
H    0     0                     745.0000000  752.85714  1.05%     -    0s
     0     0  752.85714    0    2  745.00000  752.85714  1.05%     -    0s

Explored 1 nodes (62 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 7: 745 715 700 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 7.450000000000e+02, best bound 7.450000000000e+02, gap 0.0000%

=== 最优解 ===
总利润: ₲745.00
 - 在位置 truck8 部署 Small 卡车 (容量: 100, 成本: ₲200)
 - 在位置 truck17 部署 Small 卡车 (容量: 100, 成本: ₲200)
 - 在位置 truck33 部署 Small 卡车 (容量: 100, 成本: ₲200)
 - 在位置 truck37 部署 Small 卡车 (容量: 100, 成本: ₲200)

财务明细:
总收入: ₲1545.00
总成本: ₲800.00


# 4. Sensitivity Analysis

## 4.1 Data Generation for Analysis

What to be varied:

- Fixed truck cost
- Ingredient cost
- A high-demand customer disappears

## 4.2 Sensitivity Analysis Programming

Based on the varied data, we analyze the effects of varied data in pairs. (e.g. different truck cost with 5 demand decreased)

In [33]:
df_demand

,index,name,x,y,demand
0,demand2,ReLU Realty,190.720688,72.661102,30
1,demand11,Reinforcement Learning Puppy Training,284.838551,188.794658,5
2,demand12,KKT Air Conditioning,256.572130,158.053422,50
3,demand14,George's Basic Diet Solutions,213.706568,160.537563,35
4,demand16,Rothberg Tower,129.217925,168.611018,30
5,demand17,Multimodal Distribution Center,68.025783,229.782972,5
6,demand18,Callback Cat Café,187.614487,222.641068,50
7,demand22,Vertex Tower,342.924494,231.335559,10
8,demand26,Nonlinear Thinking Center,356.281154,317.038397,5
9,demand33,Interior Point Decorating,267.133211,362.063439,40


### 4.2.1 Sensitivity Analysis of varied truck cost with decreased demand

In [34]:
data_list = []

for delta in [1,5,10]:
    # Create new demand dataframe
    df_new_demand = df_demand.copy()
    df_new_demand['demand'] = df_new_demand['demand'].where(
        df_new_demand['demand'] <= df_new_demand['demand'].mean(),
        df_new_demand['demand'] - delta
    )
    df_truck_demand_pair = generate_scaled_demand(df_truck_demand_pair,df_new_demand) # generate

    # Read data in files
    burrito_price, ingredient_cost,truck_coordinates,truck_spots,buildings,building_names,building_coordinates,demand,scaled_demand = parameter_read(df_problem,df_truck_node,df_new_demand,df_truck_demand_pair)

    for truck_cost in np.arange(0, 1000, 10):
        # solve the linear programming
        optimal_value = LP_solution(burrito_price, ingredient_cost,truck_cost,scaled_demand,truck_spots,buildings,work_name='', write_lp_file=False,show_model=False,is_show_map=False)
        # store the optimal value in dictionary
        # (x: truck cost, y: optimal value, demand: decreased demand)
        data_list.append([truck_cost,optimal_value,delta])
        
        # print(optimal_value)


NameError: name 'LP_solution' is not defined

In [ ]:
df_truck_cost_decreased_demand = pd.DataFrame(data_list, columns=['truck_cost', 'optimal_value', 'decreased_demand'])
df_truck_cost_decreased_demand

In [ ]:
# make the line graph of different truck cost , lines represent different decreased demand
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

# 使用seaborn的lineplot，hue参数自动区分不同decreased_demand
sns.lineplot(
    data=df_truck_cost_decreased_demand,
    x='truck_cost',
    y='optimal_value',
    hue='decreased_demand',
    marker='o',  # 添加数据点标记
    palette='viridis',  # 使用好看的色系
    linewidth=2.5
)

plt.title('Optimal Value vs Truck Cost by Decreased Demand')
plt.xlabel('Truck Cost')
plt.ylabel('Optimal Value')
plt.grid(True, alpha=0.3)
plt.legend(title='Decreased Demand')
plt.tight_layout()
plt.show()

### Sensitivity Analysis of varied ingredient cost with decreased demand

In [ ]:
df_demand

Truck cost needs to be reloaded for this part

In [ ]:
data_list = []

for delta in [1,5,10]:
    # Create new demand dataframe
    df_new_demand = df_demand.copy()
    df_new_demand['demand'] = df_new_demand['demand'].where(
        df_new_demand['demand'] <= df_new_demand['demand'].mean(),
        df_new_demand['demand'] - delta
    )

    df_truck_demand_pair = generate_scaled_demand(df_truck_demand_pair,df_new_demand) # generate

    # Read data in files
    burrito_price, ingredient_cost,truck_coordinates,truck_spots,buildings,building_names,building_coordinates,demand,scaled_demand = parameter_read(df_problem,df_truck_node,df_new_demand,df_truck_demand_pair)
    
    # truck cost needs to be reloaded
    truck_cost = 250

    for ingredient_cost in np.arange(0, 10, 0.1):
        # solve the linear programming
        optimal_value = LP_solution(burrito_price, ingredient_cost,truck_cost,scaled_demand,truck_spots,buildings,work_name='', write_lp_file=False,show_model=False,is_show_map=False)
        # store the optimal value in dictionary
        # (x: truck cost, y: optimal value, demand: decreased demand)
        data_list.append([ingredient_cost,optimal_value,delta])

In [ ]:
df_ingredient_cost_decreased_demand = pd.DataFrame(data_list, columns=['ingredient_cost', 'optimal_value', 'decreased_demand'])

In [ ]:
df_ingredient_cost_decreased_demand

In [ ]:
# make the line graph of different truck cost , lines represent different decreased demand
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

# 使用seaborn的lineplot，hue参数自动区分不同decreased_demand
sns.lineplot(
    data=df_ingredient_cost_decreased_demand,
    x='ingredient_cost',
    y='optimal_value',
    hue='decreased_demand',
    marker='o',  # 添加数据点标记
    palette='viridis',  # 使用好看的色系
    linewidth=2.5
)

plt.title('Optimal Value vs Ingredient Cost by Decreased Demand')
plt.xlabel('Ingredient Cost')
plt.ylabel('Optimal Value')
plt.grid(True, alpha=0.3)
plt.legend(title='Decreased Demand')
plt.tight_layout()
plt.show()